In [1]:
#$env:PYTHONPATH="src"      # PowerShell
import sys
import os

project_root = os.path.abspath("../src")
sys.path.append(project_root)

%load_ext autoreload
%autoreload
from curves.yield_curve import YieldCurve
from products.coupon_bond import CouponBond
from products.future import EquityFuture
from products.swap import InterestRateSwap
from products.option import EuropeanCall
from models.black_scholes import BlackScholesModel
from models.calibration import ImpliedVolatilitySolver




In [2]:
curve = YieldCurve(
    maturities=[1, 2, 3, 5],
    zero_rates=[0.02, 0.025, 0.03, 0.035]
)

print(curve.zero_rate(4))   # interpolation entre 3y et 5y
print(curve.discount_factor(4))


0.0325
0.8780954309205613


In [3]:
bond = CouponBond(nominal=100, coupon_rate=0.05, maturity=5, payment_frequency=1)

print("Bond price =", bond.price(curve))

Bond price = 106.76026075076305


In [4]:
fut = EquityFuture(
    spot=4000,       # S&P 500 spot
    rate=0.03,       # 3%
    dividend_yield=0.015,  # 1.5%
    maturity=0.5     # 6 months
)

print("Future price =", fut.price())


Future price = 4030.1127817781353


In [5]:
swap = InterestRateSwap(
    notional=1_000_000,
    fixed_rate=0.03,
    payment_times=[1, 2, 3]
)

print("Fixed Leg PV =", swap.pv_fixed_leg(curve))
print("Floating Leg PV =", swap.pv_floating_leg(curve))
print("Swap Value =", swap.price(curve))
print("Swap rate =", swap.swap_rate(curve))

Fixed Leg PV = 85360.77849236093
Floating Leg PV = 86068.81472877182
Swap Value = 708.0362364108878
Swap rate = 0.02328967326889742


In [6]:
call = EuropeanCall(strike=100, maturity=1.0)
model = BlackScholesModel(spot=105, rate=0.03)

vol = 0.2

print("Call price =", model.call_price(call, vol))
print("Delta =", model.call_delta(call, vol))
print("Gamma =", model.gamma(call, vol))
print("Vega =", model.vega(call, vol))


Call price = 12.638755916496294
Delta = 0.6893295440636719
Gamma = 0.01681549194942603
Vega = 37.078159748484396


In [7]:
# Market price of the call
market_price = 12.50

# Solver
solver = ImpliedVolatilitySolver(model)

iv = solver.implied_vol_call(call, market_price)

print("Implied volatility =", iv)

Implied volatility = 0.19625254622411945


In [ ]:
https://data.ecb.europa.eu/data/datasets/YC/YC.B.U2.EUR.4F.G_N_A.SV_C_YM.SR_10Y

In [ ]:
import requests
import pandas as pd
from datetime import datetime

def fetch_latest_series(series_key: str):
    """
    Fetch the series from ECB and return the latest date + value.
    """
    base_url = "https://data.ecb.europa.eu/data/datasets/YC/YC.B.U2.EUR.4F.G_N_A.SV_C_YM."
    url = f"{base_url}/{series_key}?format=sdmx-json"
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()
    # Navigate into JSON structure to find observations
    series = data["data"]["dataSets"][0]["series"]
    # Usually there is a key like "0:0:0:0:0:0:0" for the series
    series_key_inner = list(series.keys())[0]
    observations = series[series_key_inner]["observations"]
    # Get all (index,value) pairs
    date_list = data["data"]["structure"]["dimensions"]["observation"][0]["values"]
    # Build list of (date,value)
    values = []
    for idx_str, obs in observations.items():
        idx = int(idx_str)
        date = date_list[idx]["id"]
        value = obs[0]
        values.append((datetime.fromisoformat(date), float(value)))
    # Sort and get latest
    values.sort(key=lambda x: x[0])
    latest = values[-1]
    return {"date": latest[0], "value": latest[1]}



In [56]:
import sdmx
import pandas as pd

# Initialiser le client pour l'API de la BCE
ecb = sdmx.Client("ECB")

# Définir les paramètres pour la série souhaitée :
# - DATA_TYPE_FM : SR_20Y (taux spot à 20 ans)
# - INSTRUMENT_FM : G_N_A (obligations d'État, nominal, tous émetteurs notés AAA)
# - FREQ : B (quotidien, jours ouvrés)
keys = {
    'DATA_TYPE_FM': 'SR_20Y',
    'INSTRUMENT_FM': 'G_N_A',
    'FREQ': 'B',
    'REF_AREA': 'U2',
    'CURRENCY': 'EUR',
    'PROVIDER_FM': '4F',
    'PROVIDER_FM_ID': 'SV_C_YM'

}

# Récupérer les données
data_message = ecb.data('YC', key=keys,params={'startPeriod': '2025-11-24'})

# Convertir en DataFrame pandas
df = sdmx.to_pandas(data_message)

# Afficher les dernières valeurs
print("Dernières valeurs du taux spot à 20 ans (obligations d'État, zone euro, notées AAA) :")
print(df.tail())


Dernières valeurs du taux spot à 20 ans (obligations d'État, zone euro, notées AAA) :
FREQ  REF_AREA  CURRENCY  PROVIDER_FM  INSTRUMENT_FM  PROVIDER_FM_ID  DATA_TYPE_FM  TIME_PERIOD
B     U2        EUR       4F           G_N_A          SV_C_YM         SR_20Y        2025-11-24     3.298596
Name: value, dtype: float64


In [57]:
df[-1]

C:\Users\id2603\AppData\Local\Temp\ipykernel_6064\2029431133.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df[-1]


np.float64(3.2985956804)

In [ ]:
for t, df in zip(df.maturity, df.rate):

'2025-11-24'

In [59]:
import sdmx
import pandas as pd

# Initialiser le client pour l'API de la BCE
ecb = sdmx.Client("ECB")

# Periode initiale
keys = {
        'DATA_TYPE_FM': 'SR_3M',
        'INSTRUMENT_FM': 'G_N_A',
        'FREQ': 'B',
        'REF_AREA': 'U2',
        'CURRENCY': 'EUR',
        'PROVIDER_FM': '4F',
        'PROVIDER_FM_ID': 'SV_C_YM'
    }
data_message = ecb.data('YC', key=keys,params={'startPeriod': '2025-11-24'})
df = sdmx.to_pandas(data_message)

start_period = df.index[-1][-1]

# Liste des clés de maturité
MATURITIES = [0.25,0.5,0.75]+list(range(1,11))#+[25,30]
KEYS = [
    "SR_3M", "SR_6M", "SR_9M",
    "SR_1Y", "SR_2Y", "SR_3Y", "SR_4Y", "SR_5Y",
    "SR_6Y", "SR_7Y", "SR_8Y", "SR_9Y",
    "SR_10Y",# "SR_11Y", "SR_12Y", "SR_13Y", "SR_14Y", "SR_15Y", "SR_16Y", "SR_17Y", "SR_18Y", "SR_19Y", "SR_20Y", "SR_25Y", "SR_30Y"
]

# Dictionnaire pour stocker les dernières valeurs
last_values = {}

# Récupérer la dernière valeur pour chaque clé
for key in KEYS:
    keys = {
        'DATA_TYPE_FM': key,
        'INSTRUMENT_FM': 'G_N_A',
        'FREQ': 'B',
        'REF_AREA': 'U2',
        'CURRENCY': 'EUR',
        'PROVIDER_FM': '4F',
        'PROVIDER_FM_ID': 'SV_C_YM'
    }
    data_message = ecb.data('YC', key=keys,params={'startPeriod': start_period})
    df = sdmx.to_pandas(data_message)
    if not df.empty:
        last_values[key] = df[-1]



C:\Users\id2603\AppData\Local\Temp\ipykernel_6064\14459984.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  last_values[key] = df[-1]
C:\Users\id2603\AppData\Local\Temp\ipykernel_6064\14459984.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  last_values[key] = df[-1]
C:\Users\id2603\AppData\Local\Temp\ipykernel_6064\14459984.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  last_values[key] = df[-1]
C:\Users\id2603\AppDat

In [ ]:
MATURITIES = [
    "SR_3M" : 0.5, "SR_6M", "SR_9M",
    "SR_1Y", "SR_2Y", "SR_3Y", "SR_4Y", "SR_5Y",
    "SR_6Y", "SR_7Y", "SR_8Y", "SR_9Y",
    "SR_10Y",# "SR_11Y", "SR_12Y", "SR_13Y", "SR_14Y", "SR_15Y", "SR_16Y", "SR_17Y", "SR_18Y", "SR_19Y", "SR_20Y", "SR_25Y", "SR_30Y"
]

In [70]:
def get_maturity(row):
    q = 1/12
    code = row['key']
    if "M" in code:
        return int(code[3])*q
    else : 
        return int(code[3])

In [71]:

# Créer un DataFrame avec les dernières valeurs
result_df = pd.DataFrame(list(last_values.items()), columns=['key', 'rate'])
print(result_df)

       key      rate
0    SR_3M  1.934091
1    SR_6M  1.935487
2    SR_9M  1.939872
3    SR_1Y  1.946987
4    SR_2Y  1.998124
5    SR_3Y  2.076157
6    SR_4Y  2.171063
7    SR_5Y  2.275312
8    SR_6Y  2.383314
9    SR_7Y  2.490989
10   SR_8Y  2.595416
11   SR_9Y  2.694572
12  SR_10Y  2.787117


In [72]:
result_df['maturity'] = result_df.apply(get_maturity,axis=1)

In [76]:
result_df.sort_values(by="maturity")
result_df

,key,rate,maturity
0,SR_3M,1.934091,0.25
1,SR_6M,1.935487,0.50
2,SR_9M,1.939872,0.75
3,SR_1Y,1.946987,1.00
4,SR_2Y,1.998124,2.00
5,SR_3Y,2.076157,3.00
6,SR_4Y,2.171063,4.00
7,SR_5Y,2.275312,5.00
8,SR_6Y,2.383314,6.00
9,SR_7Y,2.490989,7.00


In [ ]:
dic = {result_df.loc[i,"key"] : float(result_df.loc[i,'rate']) for i in result_df.index}
dic

{'SR_3M': 1.9340910699,
 'SR_6M': 1.9354868709,
 'SR_9M': 1.9398719628,
 'SR_1Y': 1.9469869287,
 'SR_2Y': 1.9981241414,
 'SR_3Y': 2.0761572643,
 'SR_4Y': 2.1710630634,
 'SR_5Y': 2.2753116203,
 'SR_6Y': 2.3833142892,
 'SR_7Y': 2.4909886159,
 'SR_8Y': 2.5954158956,
 'SR_9Y': 2.6945720632,
 'SR_10Y': 2.7871165917}

In [77]:
result_df.to_csv('../data/yield_curve.csv')

In [84]:
list(result_df.rate)

[1.9340910699,
 1.9354868709,
 1.9398719628,
 1.9469869287,
 1.9981241414,
 2.0761572643,
 2.1710630634,
 2.2753116203,
 2.3833142892,
 2.4909886159,
 2.5954158956,
 2.6945720632,
 2.7871165917]